In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
data_skripsi=pd.read_csv('FIX_DATASKRIPSI_NEW.CSV', sep=';', encoding= 'unicode_escape')
df = data_skripsi.pivot_table(index='Item_name', columns='User_Name', values='Rating').fillna(0)
df1 = df.copy()
df1

User_Name,A,A AGUSLIMI SHAFIRA PUTRI AP,A Dian Christianti,AA,AAN AMBARYATI,AB,ABDUL WAHAB ISKANDAR,AC,ACHMAT LUTHFI YAKIM,AD,...,yuri yah,yusuf satriyo,yuyun sutaji,yuzon erman,zahra azzahra,zainul arifin,zanni erdiawan saputra,zevina Liemahassana,ziah nur aisjah,zuraidah fitri lubis
Item_name,,,,,,,,,,,,,,,,,,,,,
@HOM Premiere Timoho,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARTOTEL Suites Bianti - Yogyakarta,0.0,0.0,0.0,5.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abadi Hotel Malioboro Yogyakarta by Tritama Hospitality,5.0,0.0,0.0,3.6,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Adhisthana Hotel,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,1.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Airlangga Hotel,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Votel Maerakatja Yogyakarta,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YATS Colony,0.0,0.0,0.0,3.2,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yellow Star Ambarrukmo Hotel,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
user_index = df1.columns.tolist()

In [4]:
def get_user_rating(df, user):
  return df[df[user] > 0][user].index.tolist()

def recommend_hotels(df, df1, user, num_recommended_hotel):
  recommended_hotel = {}
  for m in df[df[user] == 0].index.tolist():
    index_df = df.index.tolist().index(m)
    predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
    recommended_hotel[m] = predicted_rating

  sorted_rm = {k: v for k, v in sorted(recommended_hotel.items(), 
      key=lambda item: item[1], reverse=True)[:num_recommended_hotel]}
  return sorted_rm

In [5]:
def hotels_recommender(df, user, num_neighbors, num_recommendation):
  df1 = df.copy()

  number_neighbors = num_neighbors

  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(df.values)
  distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

  user_index = df.columns.tolist().index(user)

  for m,t in list(enumerate(df.index)):
    if df.iloc[m, user_index] == 0:
      sim_hotels = indices[m].tolist()
      hotels_distances = distances[m].tolist()
    
      if m in sim_hotels:
        id_hotels = sim_hotels.index(m)
        sim_hotels.remove(m)
        hotels_distances.pop(id_hotels) 

      else:
        sim_hotels = sim_hotels[:num_neighbors-1]
        hotels_distances = hotels_distances[:num_neighbors-1]
            
      hotels_similarity = [1-x for x in hotels_distances]
      hotels_similarity_copy = hotels_similarity.copy()
      nominator = 0

      for s in range(0, len(hotels_similarity)):
        if df.iloc[sim_hotels[s], user_index] == 0:
          if len(hotels_similarity_copy) == (number_neighbors - 1):
            hotels_similarity_copy.pop(s)
          
          else:
            hotels_similarity_copy.pop(s-(len(hotels_similarity)-len(hotels_similarity_copy)))
            
        else:
          nominator = nominator + hotels_similarity[s]*df.iloc[sim_hotels[s],user_index]
          
      if len(hotels_similarity_copy) > 0:
        if sum(hotels_similarity_copy) > 0:
          predicted_r = nominator/sum(hotels_similarity_copy)
        
        else:
          predicted_r = 0

      else:
        predicted_r = 0
        
      df1.iloc[m,user_index] = predicted_r

  return recommend_hotels(df, df1, user,num_recommendation)

In [6]:
hotels_recommender(df, 'Afni Manopo', 15, 15)

{'Adhisthana Hotel ': 4.0,
 'Amaris Hotel Malioboro': 4.0,
 'Arte Hotel Malioboro Yogyakarta': 4.0,
 'Banana Homestay Prawirodirjan': 4.0,
 'Best City Hotel': 4.0,
 'Brothers Inn Babarsari': 4.0,
 'Burza Hotel Yogyakarta': 4.0,
 'Dem Ayem Heritage Guest House': 4.0,
 'Eastparc Hotel Yogyakarta': 4.0,
 'Eclipse Hotel': 4.0,
 'Forriz Hotel Yogyakarta': 4.0,
 'Front One Cabin Malioboro Yogyakarta': 4.0,
 'Grand Ambarrukmo Yogyakarta': 4.0,
 'Grand Kangen Hotel Urip Sumoharjo Yogyakarta': 4.0,
 'Grand Keisha Yogyakarta': 4.0}

In [7]:
hotels = data_skripsi[["Item_id", "Item_name", "Bintang_Hotel"]].drop_duplicates().reset_index(drop=True)

In [8]:
hotels.to_csv("data_hotels.csv", index=False)

In [9]:
def batch()

SyntaxError: invalid syntax (445367983.py, line 1)

In [ ]:
data_skripsi.shape

In [ ]:
round(12527/15)

In [ ]:
def batch(df, n):
    long = round(df.shape[0]/n)
    last = 0
    for i in range(n):
        if i == n-1:
            df.iloc[last:].to_csv("ratings_{}.csv".format(i), index=False)
        elif i==0:
            df.iloc[:long].to_csv("ratings_{}.csv".format(i), index=False)
        else:
            until = last + long
            df.iloc[last:until].to_csv("ratings_{}.csv".format(i), index=False)
        
        last = last + long

In [ ]:
batch(data_skripsi, 15)

In [ ]:
[ i for i in range(5)]